Import all the dependencies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import purpleair
import pandas as pd
from purpleair.network import SensorList
from purpleair.sensor import Sensor

Pull all the current data into a data frame

In [2]:
from purpleair.network import SensorList
p = SensorList()
df = p.to_dataframe(sensor_filter='all',
                    channel='parent')
df

Initialized 22,532 sensors!


,parent,lat,lon,name,location_type,pm_2.5,temp_f,temp_c,humidity,pressure,...,last_update_check,created,uptime,is_owner,10min_avg,30min_avg,1hour_avg,6hour_avg,1day_avg,1week_avg
id,,,,,,,,,,,,,,,,,,,,,
14633,None,37.275561,-121.964134,Hazelwood canary,outside,15.35,63.0,17.222222,44.0,1012.16,...,None,None,None,False,26.86,20.18,15.30,14.27,15.32,9.80
25999,None,30.053808,-95.494643,Villages of Bridgestone AQI,outside,28.83,66.0,18.888889,66.0,1016.76,...,None,None,None,False,28.59,24.91,21.38,19.03,19.33,15.75
14091,None,37.883620,-122.070087,WC Hillside,outside,14.74,62.0,16.666667,51.0,1007.30,...,None,None,None,False,12.91,11.76,11.25,16.90,23.13,16.84
108226,None,38.573703,-121.439113,"""C"" Street Air Shelter",inside,8.67,78.0,25.555556,45.0,1020.21,...,None,None,None,False,8.66,8.05,7.50,6.67,7.15,5.32
49409,None,18.759182,99.017172,"""First's Place""",outside,23.74,98.0,36.666667,27.0,982.58,...,None,None,None,False,23.21,22.44,22.37,22.62,23.29,26.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64085,None,36.785883,127.157040,청룡동행정복지센터,outside,11.74,38.0,3.333333,31.0,1018.53,...,None,None,None,False,11.37,10.69,10.44,18.15,34.14,41.38
64995,None,36.691324,126.585255,한서대학교,outside,12.57,43.0,6.111111,36.0,1009.76,...,None,None,None,False,11.17,11.31,11.21,14.74,21.31,26.66
64093,None,36.710720,126.548390,해미읍성,outside,11.39,41.0,5.000000,39.0,1020.61,...,None,None,None,False,11.01,11.05,10.97,14.91,24.71,29.71


In [27]:
se = Sensor(27395)
sensor_data = se.parent.get_historical(weeks_to_get=1,
                              thingspeak_field='secondary')[['created_at', '2.5um/dl']]
sensor_data['created_at'] = sensor_data['created_at'].dt.floor('T')
sensor_data.iloc[0]['created_at'].minute / 2

0.5

In [61]:
# This is a function that takes in a list of sensor name strings and returns a data frame with data 
# from the past week for each sensor

from datetime import datetime
from datetime import timedelta

def time_series_name(sensor_list):
    data = pd.DataFrame(columns=['created_at'])
    for sensor in sensor_list:
        sensor_index = df.index[df['name'] == sensor].to_list()[0]
        se = Sensor(sensor_index)
        sensor_data = se.parent.get_historical(weeks_to_get=1,
                              thingspeak_field='secondary')[['created_at', '2.5um/dl']]
        sensor_data['created_at'] = sensor_data['created_at'].dt.floor('T')
        # for cases where there is no data 
        if len(sensor_data) < 5000:
            continue 
        # for cases that begin at an even time 
        if (sensor_data.iloc[0]['created_at'].minute / 2) == 0:
            new_time = []
            for i in sensor_data['created_at']:
                new_time.append(i + timedelta(minutes=1))
            sensor_data['created_at'] = new_time
        data = pd.merge(data,sensor_data,how='outer',on='created_at')
    return data[~data.isnull().any(axis=1)]



# This function is the same as the one above, but takes in a list of sensor index number rather than name strings
# This version was made because the name strings can be a little finicky about spaces and such.

def time_series_index(sensor_index_list):
    data = pd.DataFrame(columns=['created_at'])
    for sensor in sensor_index_list:
        se = Sensor(sensor)
        sensor_data = se.parent.get_historical(weeks_to_get=1,
                              thingspeak_field='secondary')[['created_at', '2.5um/dl']]
        sensor_data['created_at'] = sensor_data['created_at'].dt.floor('T')
        # for cases where there is no data 
        if len(sensor_data) < 5000:
            continue 
        # for cases that begin at an even time 
        if (sensor_data.iloc[0]['created_at'].minute / 2) == 0:
            new_time = []
            for i in sensor_data['created_at']:
                new_time.append(i + timedelta(minutes=1))
            sensor_data['created_at'] = new_time
        data = pd.merge(data,sensor_data,how='outer',on='created_at')
    return data[~data.isnull().any(axis=1)]

In [33]:
test = time_series_name(["PA_II_D8B6", ' Hazelwood canary ', ' WC Hillside',])
test

,created_at,2.5um/dl_x,2.5um/dl_y,2.5um/dl
0,2021-11-24 00:01:00+00:00,2.55,1.60,6.74
1,2021-11-24 00:03:00+00:00,5.47,1.00,11.10
2,2021-11-24 00:05:00+00:00,5.16,1.34,8.18
3,2021-11-24 00:07:00+00:00,4.07,0.46,6.28
4,2021-11-24 00:09:00+00:00,4.03,1.66,4.68
...,...,...,...,...
4616,2021-11-30 23:47:00+00:00,3.93,0.53,1.92
4617,2021-11-30 23:49:00+00:00,2.62,1.60,1.86
4618,2021-11-30 23:51:00+00:00,3.40,0.24,2.05
4619,2021-11-30 23:57:00+00:00,2.35,0.09,3.18


In [34]:
test2 = time_series_index([27395, 14633, 14091, 25999])
test2

,created_at,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y
0,2021-11-24 00:01:00+00:00,2.55,1.60,6.74,3.21
1,2021-11-24 00:03:00+00:00,5.47,1.00,11.10,1.58
2,2021-11-24 00:05:00+00:00,5.16,1.34,8.18,2.20
3,2021-11-24 00:07:00+00:00,4.07,0.46,6.28,3.14
4,2021-11-24 00:09:00+00:00,4.03,1.66,4.68,1.93
...,...,...,...,...,...
4616,2021-11-30 23:47:00+00:00,3.93,0.53,1.92,3.39
4617,2021-11-30 23:49:00+00:00,2.62,1.60,1.86,2.93
4618,2021-11-30 23:51:00+00:00,3.40,0.24,2.05,1.33
4619,2021-11-30 23:57:00+00:00,2.35,0.09,3.18,4.14


Testing with larger sensor list

In [46]:
sensors

[98139,
 97279,
 27823,
 99769,
 99797,
 98633,
 51697,
 3298,
 3033,
 50567,
 27905,
 116073,
 86003,
 31025,
 31035,
 30911,
 6752,
 56377,
 85127,
 51959,
 48545,
 56059,
 30593,
 26659,
 28851,
 33033,
 28679,
 28823,
 27821,
 2386,
 3777,
 31163]

In [58]:
time_series_index([98139, 97279, 27823, 99769, 99797, 98633, 51697, 3298, 3033, 50567, 27905, 116073, 86003, 31035])

,created_at,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y


In [59]:
se = Sensor(31035)
sensor_data = se.parent.get_historical(weeks_to_get=1,
                              thingspeak_field='secondary')[['created_at', '2.5um/dl']]
sensor_data['created_at'] = sensor_data['created_at'].dt.floor('T')
sensor_data

,created_at,2.5um/dl
entry_id,,
362673,2021-11-24 22:41:00+00:00,5.27
362674,2021-11-24 22:43:00+00:00,2.31
362675,2021-11-24 22:45:00+00:00,6.05
362676,2021-11-24 22:47:00+00:00,3.78
362677,2021-11-24 22:49:00+00:00,3.69
...,...,...
367023,2021-11-30 23:50:00+00:00,2.45
367024,2021-11-30 23:52:00+00:00,1.52
367025,2021-11-30 23:54:00+00:00,1.14


In [60]:
se = Sensor(99769)
sensor_data = se.parent.get_historical(weeks_to_get=1,
                              thingspeak_field='secondary')[['created_at', '2.5um/dl']]
sensor_data['created_at'] = sensor_data['created_at'].dt.floor('T')
sensor_data

,created_at,2.5um/dl
entry_id,,
53230,2021-11-24 00:01:00+00:00,9.32
53231,2021-11-24 00:03:00+00:00,6.50
53232,2021-11-24 00:05:00+00:00,9.53
53233,2021-11-24 00:07:00+00:00,4.11
53234,2021-11-24 00:09:00+00:00,5.68
...,...,...
58248,2021-11-30 23:50:00+00:00,7.42
58249,2021-11-30 23:52:00+00:00,4.07
58250,2021-11-30 23:54:00+00:00,2.54


In [62]:
houston_data = df.loc[(df["lat"] >= 29) & (df["lat"] <= 30) & (df["lon"] >= -96) & (df["lon"] <= -94)]
outside = houston_data[houston_data['location_type'] == 'outside']
sensors = outside.index.to_list()
time_series_index(sensors)

,created_at,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,...,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y,2.5um/dl_x,2.5um/dl_y
0,2021-11-24 00:01:00+00:00,1.33,1.60,0.00,9.32,2.48,2.04,1.96,1.36,1.93,...,1.43,4.56,3.19,2.76,1.26,2.05,0.00,0.65,0.25,3.80
1,2021-11-24 00:03:00+00:00,0.72,2.81,0.00,6.50,3.70,0.38,2.50,1.16,1.64,...,2.28,3.20,1.79,1.03,0.84,1.04,0.02,0.89,1.18,1.79
2,2021-11-24 00:05:00+00:00,1.76,3.40,0.49,9.53,6.74,2.21,1.72,3.02,5.03,...,2.14,3.86,2.96,1.52,0.83,2.49,0.39,1.30,1.12,3.40
3,2021-11-24 00:07:00+00:00,0.67,2.56,0.25,4.11,4.17,1.86,1.90,0.93,3.78,...,4.04,3.17,2.00,2.10,1.43,2.54,0.02,1.36,0.72,5.13
4,2021-11-24 00:09:00+00:00,0.98,2.64,0.40,5.68,3.62,2.04,1.03,1.67,3.10,...,1.20,4.21,0.90,2.97,2.33,1.41,0.00,0.29,1.33,1.05
5,2021-11-24 00:11:00+00:00,1.03,2.29,0.10,9.68,6.18,4.90,2.44,0.78,4.00,...,1.44,1.31,1.79,2.56,0.83,2.05,0.00,0.47,0.52,2.17
6,2021-11-24 00:13:00+00:00,1.36,2.53,0.24,4.95,5.70,2.42,2.56,2.72,2.88,...,1.28,3.96,1.05,1.75,1.20,3.02,0.00,1.04,1.11,2.65
7,2021-11-24 00:15:00+00:00,1.47,2.74,0.00,6.10,3.86,2.32,0.85,1.89,3.50,...,1.44,3.76,1.72,0.62,1.79,2.83,0.00,2.35,1.51,2.21
10,2021-11-24 00:21:00+00:00,1.21,1.54,0.14,6.69,3.57,2.00,2.07,3.63,4.14,...,0.84,4.64,0.95,1.05,0.49,3.98,0.23,0.78,1.69,3.55
11,2021-11-24 00:23:00+00:00,1.49,2.26,0.27,3.97,4.53,1.47,3.24,2.27,4.88,...,2.28,1.70,0.31,1.71,1.60,3.41,0.00,0.46,0.76,2.85


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f19ace53-ffd9-4b3c-a807-6437cb6cda39' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>